In [ ]:
import csv
import json
import pandas as pd
import google.generativeai as genai

try:
    genai.configure(api_key="chiavAPI") 
except ValueError as e:
    print(f"Error configuring API key: {e}")
    exit()

input_file = './joe_rogan_episodes_final.csv'
output_file = './gem_data/ospiti_info_with_descriptions_3.csv'
model_name = 'gemini-2.5-pro-preview-06-05'

try:
    model = genai.GenerativeModel(model_name)
except Exception as e:
    print(f"Error creating GenerativeModel: {e}")
    print(f"Ensure the model name '{model_name}' is correct and available.")
    exit()

csv_headers = [
    'video_id',
    'Nome',
    'Nazionalità',
    'Sesso',
    'Età',
    'Professione', 
    'Notorietà',
    'Numero di follower social',
    'Topic video',
]

processed_names_cache = {}

try:
    df = pd.read_csv(input_file)
    print(f"Found {len(df)} episodes in {input_file}.")
    
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(csv_headers)

        print(f"Processing episodes and writing to {output_file}...")

        for i, row in df.iterrows():
            video_id = row['video_id']
            nome = row['guest']
            description = row['description'] if pd.notna(row['description']) else "Nessuna descrizione disponibile"
            
            print(f"Processing {i+1}/{len(df)}: {nome} (Video ID: {video_id})...")
            
            row_data = {header: '' for header in csv_headers}
            row_data['video_id'] = video_id
            row_data['Nome'] = nome
            
            cache_key = f"{nome}_{video_id}"
            if cache_key in processed_names_cache:
                print(f"  Cache hit for {nome}. Reusing previous result.")
                cached_data = processed_names_cache[cache_key]
                for key, value in cached_data.items():
                    if key in row_data:
                        row_data[key] = value
                csv_writer.writerow([row_data.get(header, '') for header in csv_headers])
                continue

            print(f"  Cache miss for {nome}. Calling API...")
            
            prompt= f"""   Fornisci informazioni di {nome} (ospite del podcast di Joe Rogan) in formato JSON. 

                            Informazioni aggiuntive dall'episodio (potrebbero non essere disponibili):
                            Link del video: https://www.youtube.com/watch?v={video_id}
                            Descrizione del video: {description}

                            Il JSON deve avere le seguenti 'chiavi': (possibili valori) = 
                            'Nome': (nome fornito), 
                            'Nazionalità': (es. Europa, America, Asia.),
                            'Sesso': (M, F),
                            'Età': (#), 
                            'Professione': La più importante. È fondamentale standardizzarla. Ad esempio, invece di avere "Sviluppatore", "Programmatore", "Ingegnere Informatico", raggruppale in "Tech/IT". 
                            'Notorietà': Creare una scala consistente. Esempio di scala: Esperto di Nicchia: Noto solo nel suo campo specifico. Influencer di Settore: Ha un seguito consolidato sulla sua piattaforma principale. Personaggio Pubblico: Noto al grande pubblico anche al di fuori del suo settore,
                            'Follower social': Il numero di follower sulla sua piattaforma principale.
                            'Topic video': Argomento Principale del Video (Categorica). Di cosa si è parlato? (es. "AI Generativa", "Crescita Personale", "Marketing Digitale"). Questo solo se riesci a recuperarlo

                            - Usa il valore JSON `null` per le informazioni che non riesci a trovare dopo una ricerca approfondita.
                            - Considera le informazioni dalla descrizione dell'episodio per contestualizzare meglio la persona.
                            - Rispondi SOLO con il JSON valido, senza testo introduttivo o conclusivo, nemmeno ```json ```."""
            
            try:
                response = model.generate_content(prompt)

                if response.parts:
                    json_string = response.text
                    if json_string.startswith("```json"):
                        json_string = json_string[7:]
                    if json_string.endswith("```"):
                        json_string = json_string[:-3]
                    json_string = json_string.strip()
                    
                    try:
                        parsed_json = json.loads(json_string)
                        if isinstance(parsed_json, dict):
                            row_data['Nazionalità'] = parsed_json.get('Nazionalità', 'null')
                            row_data['Sesso'] = parsed_json.get('Sesso', 'null')
                            row_data['Età'] = parsed_json.get('Età', 'null')
                            row_data['Professione'] = parsed_json.get('Professione', 'null')
                            row_data['Notorietà'] = parsed_json.get('Notorietà', 'null')
                            row_data['Numero di follower social'] = parsed_json.get('Follower social', 'null')
                            row_data['Topic video'] = parsed_json.get('Topic video', 'null')
                            
                            processed_names_cache[cache_key] = parsed_json
                        else:
                            raise json.JSONDecodeError("API response was not a JSON object", json_string, 0)

                    except json.JSONDecodeError as json_e:
                        print(f"  Errore: Impossibile decodificare il JSON per {nome}. Risposta: {json_string[:100]}... Errore: {json_e}")

                else:
                    print(f"  Warning: No content generated for {nome}. Response: {response.prompt_feedback}")

            except Exception as api_e:
                print(f"  Errore durante la chiamata API per {nome}: {api_e}")

            csv_writer.writerow([row_data.get(header, '') for header in csv_headers])

        print("Processing complete.")

except FileNotFoundError:
    print(f"Errore: Il file di input '{input_file}' non è stato trovato.")
except Exception as e:
    print(f"Si è verificato un errore generale: {e}")